# Initialize database

In [ ]:
import obi_one as obi
virtual_lab_id=obi.LAB_ID_STAGING_TEST
project_id=obi.PROJECT_ID_STAGING_TEST

# entitycore_api_url = "http://127.0.0.1:8000"
entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"

# Initialize the database
obi.db.initialize(virtual_lab_id=virtual_lab_id, 
                    project_id=project_id,
                    entitycore_api_url=entitycore_api_url
                    )

db_client = obi.db.client

# Find a morphology using entitysdk

In [ ]:
from entitysdk.models.cell_morphology import (
    CellMorphology,
)
morphologies = db_client.search_entity(
    entity_type=CellMorphology, query={}, limit=10
).all()

morphology = morphologies[0]

# Find a morphology using obi-one

In [ ]:
morphologies = obi.CellMorphologyFromID.find(db_client=db_client)
for morphology in morphologies:
    if morphology.assets:
        if len(morphology.assets):
            break

# Fetch a morphology using obi-one

In [ ]:
obi.CellMorphologyFromID.fetch(entity_id=morphology.id, db_client=db_client)

# Create a CellMorphologyFromID + access its properties

In [ ]:
morphology_from_id = obi.CellMorphologyFromID(id_str=str(morphology.id))

morphology_from_id.entity(db_client=db_client)
morphology_from_id.swc_file_content(db_client=db_client)
morphology_from_id.neurom_morphology(db_client=db_client)
# morphology_from_id.morphio_morphology(db_client=db_client)

## Create a CellMorphologyFromID object and pass it to the MorphologyMetricsScanConfig for GridScan execution

In [ ]:
# Create a morphology object (this will fetch a CellMorphology object and create attributes for every parameter of CellMorphology)
morphology_from_id = obi.CellMorphologyFromID(id_str=str(morphology.id))

# Create a MorphologyMetricsExampleForm.initialize object
initialize = obi.MorphologyMetricsScanConfig.Initialize(morphology=[morphology_from_id, morphology_from_id])

# Create a MorphologyMetricsExampleForm object with the initialize object
scan_config = obi.MorphologyMetricsScanConfig(initialize=initialize)

# Create the grid scan object
scan = obi.GridScan(form=scan_config,
                    coordinate_directory_option="ZERO_INDEX",
                    output_root="../../../../obi-output/morphology_test/grid_scan")

# Run the scan
scan.execute(processing_method="run", db_client=db_client)  # , data_postprocessing_method="data"


In [ ]:
obi.Circuit